# Debugging in DSC2

## Module executable and generated script

As seen from previous tutorials, the [module executables in DSC](dsc_result.html) are in fact incomplete because the lack input data and parameter specifications to run. Under the hood, DSC2 generates R, Python or Shell scripts based on module executable scripts, adding to it specification of module parameters and pipeline variables, then executes the complete script. This generated script is subject to two types of errors:

1. Inconsistency between DSC interface and module executables
2. Error from module executables as provided
 
In the presence of errors DSC will fail. However the automatically generated script will be saved to disk and will be pointed to you as part of the error prompt on the command console so that you can reproduce and fix the bugs. 

Here we look at example from [location parameter estimations example](https://github.com/stephenslab/dsc2/tree/master/vignettes/one_sample_location). We undermine the setup to demonstrate the 2 types of errors we have discussed.

## Buggy DSC interface

To demonstrate inconsistency between interface and module we change the first block of DSC script from:

```
normal, t: rnorm.R, rt.R
    seed: R(1:5)
```

to:

```
normal, t: rnorm.R, rt.R
    weed: R(1:5)
```

creating a parameter `weed`, a typo of `seed`, that does not exist in `rnorm.R` and `rt.R`. Consequently the required `seed` variable for these routines will not be provided by DSC interface. We save this configuration file as `settings_error_v1.dsc` and run it:

In [1]:
%cd ~/GIT/dsc2/vignettes/one_sample_location

/home/gaow/GIT/dsc2/vignettes/one_sample_location

In [2]:
! dsc settings_error_v1.dsc --skip none

INFO: Checking R library dscrutils@stephenslab/dsc2/dscrutils ...
INFO: DSC script exported to dsc_result.html
INFO: Constructing DSC from settings_error_v1.dsc ...
INFO: Building execution graph & running DSC ...
DSC:  18%|███████                                | 2/11 [00:01<00:05,  1.60it/s]
ERROR: [3808530b-4117-459c-990c-ab31d5074b42]: [normal_0]: 
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
script_6123501901101114663 in <module>
      ## END code by DSC2
      
----> """, workdir = './', stderr = None, stdout = None)

RuntimeError: Failed to execute commmand Rscript --default-packages=datasets,methods,utils,stats,grDevices,graphics  /home/gaow/GIT/dsc2/vignettes/one_sample_location/.sos/normal_0_0_652e6bc3.R (ret=1, workdir=/home/gaow/GIT/dsc2/vignettes/one_sample_location)
[49bb3658-fb20-4736-b8d4-1d9c2ee0c6f4]: [t_0]: 
------------------------------------------------------

The error message is a bit wordy but the part most relevant to debugging are highlighted in green color on your terminal. In this case multiple generated scripts have failed, but the cause is likely the same. You can try to resolve the problem starting from the last error message and see if the fix will help with other errors.

You can also see that a file called `dsc_result.scripts.html` is generated. It bundles generated scripts from all module instances involved in the benchmark up to the point the errors occur.

### Debug with the problematic script
As prompted in the error message, let's try to execute `.sos/t_0_0_b8a69d4f.R`:

In [3]:
! Rscript .sos/t_0_0_b8a69d4f.R

Error in set.seed(seed) : object 'seed' not found
Execution halted


Okey so what's going on? Let's take a look at this script:

In [5]:
%preview .sos/t_0_0_b8a69d4f.R -n -l -1

> .sos/t_0_0_b8a69d4f.R (337 B):

## r script UUID: c218f56b48d262e5
## BEGIN code by DSC2
n <- 1000
true_mean <- 0
weed <- 1
DSC_4AAF65D6_tic_pt <- proc.time()
## END code by DSC2
set.seed(seed)
x=true_mean+rt(n,df=2)
## BEGIN code by DSC2
saveRDS(list(x=x, true_mean=true_mean, DSC_TIMER = proc.time() - DSC_4AAF65D6_tic_pt), 'dsc_result/t_1.rds')
## END code by DSC2



It looks like `set.seed()` looks for variable `seed`, which was not provided by DSC. We know from the [DSC script](https://github.com/stephenslab/dsc2/blob/master/vignettes/one_sample_location/settings_error_v1.dsc) that we had set `weed` not `seed`. So obviously there is a glitch on the interface which can be fixed by changing `weed` to `seed` in the DSC script.

In practice when you identify problems that does not have an obvious fix such as this one, you should load the problematic script `.sos/t_0_0_b8a69d4f.R` to some interactive session, try to debug and fix the script from there (do not worry about changing this script), test until it works. Then you should have learned where the problem is, and you can apply what you've learned to patching DSC script or the original module code. *Fixing these generated scripts is not enough as they will be overwritten next time DSC runs*.

## Buggy module script

Now let's undermine the module code instead. We change `R/methods/mean.R` from:

```r
mean = mean(x)
```

to 

```r
mean = 'meow'
```

As you can probably tell, this will not result an error for this module, but may impact downstream ones. So let's find it out!

In [6]:
! dsc settings_error_v2.dsc

INFO: DSC script exported to dsc_result.html
INFO: Constructing DSC from settings_error_v2.dsc ...
INFO: Building execution graph & running DSC ...
DSC:  91%|██████████████████████████████████▌   | 10/11 [00:08<00:00,  1.22it/s]
ERROR: [01278859-93de-4e0c-8126-f4af09cf28eb]: [mse_0]: 
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
script_4384414397566502935 in <module>
      ## END code by DSC2
      
----> """, workdir = './', stderr = None, stdout = None)

RuntimeError: Failed to execute commmand Rscript --default-packages=datasets,methods,utils,stats,grDevices,graphics  /home/gaow/GIT/dsc2/vignettes/one_sample_location/.sos/mse_0_0_c3a61422.R (ret=1, workdir=/home/gaow/GIT/dsc2/vignettes/one_sample_location)
[1c24f16d-7e75-4161-bbd6-97c3e35239bb]: [mse_0]: 
---------------------------------------------------------------------------
RuntimeError                              Traceb

Same as before, we run:

In [7]:
! Rscript .sos/mse_0_0_93b09896.R

Error in mean_est - true_mean : non-numeric argument to binary operator
Execution halted


and we examine the code by adding a couple of `print` commands before the line in question:

In [10]:
## BEGIN code by DSC2
DSC_1589D5E6 <- list()
input.files <- c('dsc_result/t_1.rds','dsc_result/t_1_mean_1.rds')
for (i in 1:length(input.files)) DSC_1589D5E6 <- dscrutils:::merge_lists(DSC_1589D5E6, readRDS(input.files[i]))
mean_est <- DSC_1589D5E6$mean
true_mean <- DSC_1589D5E6$true_mean
DSC_1589D5E6_tic_pt <- proc.time()
## END code by DSC2

print(mean_est)
print(true_mean)
mse = (mean_est-true_mean)^2


## BEGIN code by DSC2
saveRDS(list(mse=mse, DSC_TIMER = proc.time() - DSC_1589D5E6_tic_pt), 'dsc_result/t_1_mean_1_mse_1.rds')
## END code by DSC2

[1] "meow"
[1] 0


ERROR: Error in mean_est - true_mean: non-numeric argument to binary operator


It looks like `mean_est`, instead of being a number, is a string `meow`. Therefore the line `mse = (mean_est-true_mean)^2` is doomed to fail. We know from the DSC script that `mean_est` comes from an upstream module ensemble `estimate`, that somehow returned `meow` as output. We can also see from this script that the variable comes from one of `'dsc_result/t_1.rds','dsc_result/t_1_mean_1.rds'`. From our knowledge in the DSC script we know it should come from `'dsc_result/t_1_mean_1.rds'` the `mean` module. Therefore there is nothing we can do in the current script to fix the problem.

### Debugging with upstream scripts

Now we want to find out what has produced the problematic `mean_est` variable. That is, we want to find the module instance that generated `'dsc_result/t_1_mean_1.rds'` from which `mean_est` is loaded. However the problematic script above did not tell us where this data comes from. This leads us to examine other scripts in the benchmark that leads to the error, in [`dsc_result.scripts.html`](dsc_result.scripts.html).

You can open up [this file](dsc_result.scripts.html) in your web browser, and *search* (`ctrl-F`) for the phrase `dsc_result/t_1_mean_1.rds`. This leads you to the following: 

![debug-1](../img/debug-tips-1.png)

Unsurprisingly, running these code by itself produces no error message -- this is expected because otherwise DSC would have failed at this stage. However we know from previous investigation that the output does not look right. For this example we can spot instantly that the `meow` string somehow &#128049; replaced the code for location parameter estimation; and we can fix it easily. In practice, one may need to load both this code chunk and the previous `.sos/mse_0_0_93b09896.R` to an interactive session, fix the code until `.sos/mse_0_0_93b09896.R` works, and apply the knowledge learned to the original module `R/method/mean.R`. 

## Debug with `--target` and `--truncate`
It is highly recommanded that these options be used to check the correctness of DSC benchmark as you develop them. Do not run the entire benchmark unless you have checked it bit by bit. `--truncate` will only run one instance from a module, thus enables running a fraction of DSC benchmark relatively quickly to ensure everthing is correct. Additionally it is suggested that a benchmark be executed module by module using `--target`: the next module or module ensemble is only attedmpted when there is no obvious problem with the previous module. For example for a DSC benchmark:

```
DSC:
  define: 
      simulate: normal, t
      estimate: median, mean
  run: simulate * estimate * mse
```
When the benchmark is tested for the first time, one should at least run the following sequentially:

```
dcs file.dsc --target normal --truncate
dcs file.dsc --target t --truncate
dcs file.dsc --target "normal * mean" --truncate
dcs file.dsc --target "normal * median" --truncate
dcs file.dsc --target "normal * median * mse" --truncate
```

This will ensure all module are tested for at least one instance. Then you can run the benchmark knowing that the chances of errors has already been reduced.

```
dcs file.dsc 
```

## Log files
The run-time information are kept in greater detail in `*.log` file, when used with verbosity level greater than `2`. Though typically unnecessary, if you still cannot fix issues at this point it may help re-running benchmark with `-v 4` and look into the log file for more run-time information to help performing diagnostics.